In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import itertools

# JSON API
import requests
import json

# data visualization
import matplotlib
import seaborn as sns
import statsmodels.api as sm

%matplotlib inline

# ignore warnings
import warnings
warnings.filterwarnings("ignore")

from acquire import tsa_acquire_all
from acquire import peekatdata
from acquire import missing_values_col
#epare import data_prepped

In [2]:
df = tsa_acquire_all()

Checking for existing items.csv...
Reading items from local csv...

Checking for existing stores.csv...
Reading stores from local csv...

Checking for existing sales.csv...
Reading sales from local csv...

Writing final df.csv...

Finished writing final df.csv.  Final df done.

HEADS UP: finished all tsa acquire.  Full Stop!

Final raw df now ready to begin data preparations.





In [3]:
df.tail(3)

,item_id,sale_amount,sale_date,sale_id,store_id,item_brand,item_name,item_price,item_upc12,item_upc14,store_address,store_city,store_state,store_zipcode
912997,50,74.0,"Fri, 29 Dec 2017 00:00:00 GMT",912998,10,Choice,Choice Organic Teas Black Tea Classic Black - ...,5.2,47445919221,47445919221,8503 NW Military Hwy,San Antonio,TX,78231
912998,50,62.0,"Sat, 30 Dec 2017 00:00:00 GMT",912999,10,Choice,Choice Organic Teas Black Tea Classic Black - ...,5.2,47445919221,47445919221,8503 NW Military Hwy,San Antonio,TX,78231
912999,50,82.0,"Sun, 31 Dec 2017 00:00:00 GMT",913000,10,Choice,Choice Organic Teas Black Tea Classic Black - ...,5.2,47445919221,47445919221,8503 NW Military Hwy,San Antonio,TX,78231


In [4]:
def time_convert(df):
    datetime_format = '%a, %d %b %Y %H:%M:%S %Z'
    df['sale_date'] = pd.to_datetime(df.sale_date,format=datetime_format,utc=True)
    return df
time_convert(df)

,item_id,sale_amount,sale_date,sale_id,store_id,item_brand,item_name,item_price,item_upc12,item_upc14,store_address,store_city,store_state,store_zipcode
0,1,13.0,2013-01-01 00:00:00+00:00,1,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253
1,1,11.0,2013-01-02 00:00:00+00:00,2,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253
2,1,14.0,2013-01-03 00:00:00+00:00,3,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253
3,1,13.0,2013-01-04 00:00:00+00:00,4,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253
4,1,10.0,2013-01-05 00:00:00+00:00,5,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253
5,1,12.0,2013-01-06 00:00:00+00:00,6,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253
6,1,10.0,2013-01-07 00:00:00+00:00,7,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253
7,1,9.0,2013-01-08 00:00:00+00:00,8,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253
8,1,12.0,2013-01-09 00:00:00+00:00,9,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253
9,1,9.0,2013-01-10 00:00:00+00:00,10,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253


In [5]:
def time_convert(df):
#    df['date_original'] = pd.to_datetime(df.sale_date)
    df['year'] = df.sale_date.dt.year
    df['quarter'] = df.sale_date.dt.quarter
    df['month'] = df.sale_date.dt.month
    df['day'] = df.sale_date.dt.day
    df['weekday'] = df.sale_date.dt.day_name().str[:3]
    df['is_weekend'] = df.weekday.str.startswith('Sat') | df.weekday.str.startswith('Sun')    
    return df

time_convert(df)

,item_id,sale_amount,sale_date,sale_id,store_id,item_brand,item_name,item_price,item_upc12,item_upc14,store_address,store_city,store_state,store_zipcode,year,quarter,month,day,weekday,is_weekend
0,1,13.0,2013-01-01 00:00:00+00:00,1,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,2013,1,1,1,Tue,False
1,1,11.0,2013-01-02 00:00:00+00:00,2,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,2013,1,1,2,Wed,False
2,1,14.0,2013-01-03 00:00:00+00:00,3,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,2013,1,1,3,Thu,False
3,1,13.0,2013-01-04 00:00:00+00:00,4,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,2013,1,1,4,Fri,False
4,1,10.0,2013-01-05 00:00:00+00:00,5,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,2013,1,1,5,Sat,True
5,1,12.0,2013-01-06 00:00:00+00:00,6,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,2013,1,1,6,Sun,True
6,1,10.0,2013-01-07 00:00:00+00:00,7,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,2013,1,1,7,Mon,False
7,1,9.0,2013-01-08 00:00:00+00:00,8,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,2013,1,1,8,Tue,False
8,1,12.0,2013-01-09 00:00:00+00:00,9,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,2013,1,1,9,Wed,False
9,1,9.0,2013-01-10 00:00:00+00:00,10,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,2013,1,1,10,Thu,False


In [18]:
def improve_sales_data(df):
    df['sale_total'] = df.sale_amount * df.item_price
    return df.rename(columns={'sale_amount': 'quantity'})
improve_sales_data(df)

,item_id,quantity,sale_date,sale_id,store_id,item_brand,item_name,item_price,item_upc12,item_upc14,...,store_city,store_state,store_zipcode,year,quarter,month,day,weekday,is_weekend,sale_total
0,1,13.0,2013-01-01 00:00:00+00:00,1,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,...,San Antonio,TX,78253,2013,1,1,1,Tue,False,10.92
1,1,11.0,2013-01-02 00:00:00+00:00,2,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,...,San Antonio,TX,78253,2013,1,1,2,Wed,False,9.24
2,1,14.0,2013-01-03 00:00:00+00:00,3,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,...,San Antonio,TX,78253,2013,1,1,3,Thu,False,11.76
3,1,13.0,2013-01-04 00:00:00+00:00,4,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,...,San Antonio,TX,78253,2013,1,1,4,Fri,False,10.92
4,1,10.0,2013-01-05 00:00:00+00:00,5,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,...,San Antonio,TX,78253,2013,1,1,5,Sat,True,8.40
5,1,12.0,2013-01-06 00:00:00+00:00,6,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,...,San Antonio,TX,78253,2013,1,1,6,Sun,True,10.08
6,1,10.0,2013-01-07 00:00:00+00:00,7,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,...,San Antonio,TX,78253,2013,1,1,7,Mon,False,8.40
7,1,9.0,2013-01-08 00:00:00+00:00,8,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,...,San Antonio,TX,78253,2013,1,1,8,Tue,False,7.56
8,1,12.0,2013-01-09 00:00:00+00:00,9,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,...,San Antonio,TX,78253,2013,1,1,9,Wed,False,10.08
9,1,9.0,2013-01-10 00:00:00+00:00,10,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,...,San Antonio,TX,78253,2013,1,1,10,Thu,False,7.56


### the rename above DOES NOT WORK!!!! FIND OUT WHY!!!

In [17]:
df.columns

Index(['item_id', 'sale_amount', 'sale_date', 'sale_id', 'store_id',
       'item_brand', 'item_name', 'item_price', 'item_upc12', 'item_upc14',
       'store_address', 'store_city', 'store_state', 'store_zipcode', 'year',
       'quarter', 'month', 'day', 'weekday', 'is_weekend', 'sale_total'],
      dtype='object')

look at github to find Zach's code... I'm missng some steps here.

In [19]:
def ():
    df.groupby('weekday')[['sale_total','sale_amount']].agg(['median', 'sum'])
    return df

sale_total               sale_amount           
            median           sum      median        sum
weekday                                                
Fri         250.43  3.868773e+07        50.0  7198021.0
Mon         186.76  2.896118e+07        38.0  5385853.0
Sat         266.97  4.116177e+07        53.0  7655482.0
Sun         282.47  4.359486e+07        57.0  8109705.0
Thu         234.52  3.628727e+07        47.0  6749880.0
Tue         218.66  3.383954e+07        44.0  6293481.0
Wed         218.92  3.392832e+07        44.0  6312090.0

In [14]:
df.head(20).sale_total.diff()

0      NaN
1    -1.68
2     2.52
3    -0.84
4    -2.52
5     1.68
6    -1.68
7    -0.84
8     2.52
9    -2.52
10    0.00
11   -1.68
12    2.52
13    1.68
14   -5.88
15    1.68
16    7.56
17   -7.56
18    9.24
19   -2.52
Name: sale_total, dtype: float64

In [ ]:
dates = pd.Series(df.index)

In [ ]:
working with dates

In [20]:
import pandas as pd

In [21]:
date_index = pd.date_range(start='2019-04-18 13:30:00', freq='30min', periods=10)

### this is broken below... SEE ZACH'S 'WORKING WITH DATES' NOTEBOOK (on git hub, codeup ds classroom/time_series)  FOR THE DETAILS AND PROPER SYNTAX!!

### Many good examples here!!

In [22]:
df = pd.DataFrame(dict(fatige=range(10)))

,item_id,sale_amount,sale_date,sale_id,store_id,item_brand,item_name,item_price,item_upc12,item_upc14,...,store_city,store_state,store_zipcode,year,quarter,month,day,weekday,is_weekend,sale_total
0,1,13.0,2013-01-01 00:00:00+00:00,1,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,...,San Antonio,TX,78253,2013,1,1,1,Tue,False,10.92
1,1,11.0,2013-01-02 00:00:00+00:00,2,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,...,San Antonio,TX,78253,2013,1,1,2,Wed,False,9.24
2,1,14.0,2013-01-03 00:00:00+00:00,3,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,...,San Antonio,TX,78253,2013,1,1,3,Thu,False,11.76
3,1,13.0,2013-01-04 00:00:00+00:00,4,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,...,San Antonio,TX,78253,2013,1,1,4,Fri,False,10.92
4,1,10.0,2013-01-05 00:00:00+00:00,5,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,...,San Antonio,TX,78253,2013,1,1,5,Sat,True,8.40


In [ ]:
def new_index(df):
    df = df.set_index('sale_date')
    return df

new_index(df)